
## About: 


Real-world data is rarely clean and homogeneous. It is often said that 80% of data analysis is spent on the process of cleaning and preparing
the data (Dasu and Johnson 2003). Data preparation is not just a first step, but must be
repeated many over the course of analysis as new problems come to light or new data is
collected. In this notebook we showcase various data cleaning methods:  

## Topics: 

- Handle uninformative and duplicative values (involves outlier removal, Gausian/Medial Filter,  Exponential smoothing etc)

- Fill in missing values (Mean, regression models and statistical distributions)

- Outlier Detection

### References: 

This notebook contains excerpts and ideas from the following: 

 - Python Data Science Handbook by Jake VanderPlas
 - Handle Missing values: https://jakevdp.github.io/PythonDataScienceHandbook/03.04-missing-values.html
 - Data Science design Manual



### Further reading:

- Tidy data: http://vita.had.co.nz/papers/tidy-data.pdf 
- Best Practices in Data Cleaning: A Complete Guide to Everything You Need to Do Before and After Collecting Your Data
- Data Wrangling with Python by Jacqueline Kazil, Katharine Jarmul
- Clean Data by Megan Squire 
- Python Data Cleaning Cookbook - Modern techniques and Python tools to detect and remove dirty data and extract key insights
By: Michael Walker

## Missing values 

Not all data sets are complete. An important aspect of data cleaning is identifying fields for which data isn’t there, and then properly compensating for
them. 

Numerical data sets expect a value for every element in a matrix. Setting
missing values to zero is tempting, but generally wrong, because there is always
some ambiguity as to whether these values should be interpreted as data or not.

The danger with using nonsense values as not-data symbols is that they
can get misinterpreted as data when it comes time to build models. A linear
regression model trained to predict salaries from age, education, and gender will
have trouble with people who refused to answer the question.

Generally, they revolve around one of two strategies: using a mask that globally indicates missing values, or choosing a sentinel value that indicates a missing entry.

Pandas chose to use sentinels for missing data, and further chose to use two already-existing Python null values: the special floating-point NaN value, and the Python None object




NaN and None both have their place, and Pandas is built to handle the two of them nearly interchangeably, converting between them where appropriate:


### None: Pythonic missing data




In [ ]:
import numpy as np
import pandas as pd

In [ ]:
vals1 = np.array([1, None, 3, 4])
vals1

array([1, None, 3, 4], dtype=object)

This dtype=object means that the best common type representation NumPy could infer for the contents of the array is that they are Python objects. While this kind of object array is useful for some purposes, any operations on the data will be done at the Python level, with much more overhead than the typically fast operations seen for arrays with native types:



In [ ]:
for dtype in ['object', 'int']:
    print("dtype =", dtype)
    %timeit np.arange(1E6, dtype=dtype).sum()
    print()

dtype = object
63.6 ms ± 1.08 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)

dtype = int
1.25 ms ± 7.06 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)



### NaN: Missing numerical data

NaN (acronym for Not a Number), is different; it is a special floating-point value recognized by all systems that use the standard IEEE floating-point representation:

In [ ]:
vals2 = np.array([1, np.nan, 3, 4]) 
vals2.dtype

dtype('float64')

Notice that NumPy chose a native floating-point type for this array: this means that unlike the object array from before, this array supports fast operations pushed into compiled code. You should be aware that NaN is a bit like a data virus–it infects any other object it touches. Regardless of the operation, the result of arithmetic with NaN will be another NaN:



In [ ]:
1 + np.nan


nan

In [ ]:
0 *  np.nan


nan

### Strategies for filling values: 

- Heuristic-based imputation: Given sufficient knowledge of the underlying
domain, we should be able to make a reasonable guess for the value of
certain fields. 

- Mean value imputation: Using the mean value of a variable as a proxy
for missing values is generally sensible.

- Random value imputation: Another approach is to select a random value
from the column to replace the missing value.

- Imputation by nearest neighbor: What if we identify the complete record
which matches most closely on all fields present, and use this nearest
neighbor to infer the values of what is missing?

- Imputation by interpolation: More generally, we can use a method like
linear regression  to predict the values of the target column, given the other fields in the record.

----------------------------------


It is quite common to have not-a-number (NaN) values in your data set. 
To be able to operate on a data set with statistical methods, you’ll first need to clean up the data. 

Pandas treats None and NaN as essentially interchangeable for indicating missing or null values. To facilitate this convention, there are several useful methods for detecting, removing, and replacing null values in Pandas data structures. They are:

The fillna and dropna Pandas functions are a convenient way to replace the NaN values with something more representative for your data set, for example, a zero, or to remove the rows with NaN values from the data frame.

All methods: 

- isnull(): Generate a boolean mask indicating missing values
- notnull(): Opposite of isnull()
- dropna(): Return a filtered version of the data
- fillna(): Return a copy of the data with missing values filled or imputed



In [ ]:
data = pd.Series([1, np.nan, 'hello', None])


In [ ]:
data.isnull()


0    False
1     True
2    False
3     True
dtype: bool

In [ ]:
data.dropna() 

0        1
2    hello
dtype: object

In [ ]:
df = pd.DataFrame([[1,      np.nan, 2],
                   [2,      3,      5],
                   [np.nan, 4,      6]])
df

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


In [ ]:
#By default, dropna() will drop all rows in which any null value is present:



df.dropna()


,0,1,2
1,2.0,3.0,5


In [ ]:
# Alternatively, you can drop NA values along a different axis; axis=1 drops all columns containing a null value:

df.dropna(axis='columns')



,2
0,2
1,5
2,6


But this drops some good data as well; you might rather be interested in dropping rows or columns with all NA values,

 or a majority of NA values. 

This can be specified through the how or thresh parameters, which allow fine control of the number of nulls to allow through.

The default is how='any', such that any row or column (depending on the axis keyword) containing a null value will be dropped. You can also specify how='all', which will only drop rows/columns that are all null values:

In [ ]:

#replace all instances of NaN in the dataframe with zero 

df.fillna(0)

We can specify a forward-fill to propagate the previous value forward:

#data.fillna(method='ffill')

Or we can specify a back-fill to propagate the next values backward:

#data.fillna(method='bfill')

df.fillna(method='ffill', axis=1)



SyntaxError: invalid syntax (<ipython-input-13-f9d613fe495e>, line 5)

## Outlier Detection

Outlier elements are often created by data entry mistakes, as apparently was
the case here. They can also result from errors in scraping, say an irregularity
in formatting causing a footnote number to be interpreted as a numerical value.
Just because something is written down doesn’t make it correct. As with the
dinosaur example, a single outlier element can lead to major misinterpretations.

General sanity checking requires looking at the largest and smallest values
in each variable/column to see whether they are too far out of line. This can
best be done by plotting the frequency histogram and looking at the location of
the extreme elements. Visual inspection can also confirm that the distribution
looks the way it should, typically bell-shaped.